# Payoff d'un straddle long sur SPY

Ce TP illustre le payoff d'un straddle européen (call + put au même strike) sur SPY.
Objectifs pédagogiques :
1. Observer l'évolution du sous-jacent SPY sur un an pour fixer un spot de référence.
2. Définir le payoff théorique du straddle (somme du payoff call et du payoff put au même strike).
3. Visualiser la courbe de payoff en fonction du strike autour du spot de référence.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
from pathlib import Path

plt.style.use("seaborn-v0_8-darkgrid")

def fetch_spy_history(period="1y", interval="1d") -> pd.Series:
    """Récupère les prix SPY (cache local puis fallback yfinance)."""
    cache_path = Path("notebooks/GPT/_cache_spy_close.csv")
    if cache_path.exists():
        try:
            cached = pd.read_csv(cache_path, index_col=0, parse_dates=True)
            if not cached.empty and "Close" in cached.columns:
                return cached["Close"]
        except Exception:
            pass
    data = yf.download("SPY", period=period, interval=interval, progress=False)
    if data.empty or "Close" not in data:
        raise RuntimeError("Impossible de récupérer les prix SPY")
    close = data["Close"]
    try:
        cache_path.parent.mkdir(parents=True, exist_ok=True)
        close.to_csv(cache_path, index_label="date")
    except Exception:
        pass
    return close

# Récupération des prix de clôture (dernier close pour spot_ref)
close_spy = fetch_spy_history()
spot_ref = float(close_spy.iloc[-1])



def payoff_call(spot: float, strike: float) -> float:
    return max(spot - strike, 0.0)

def payoff_put(spot: float, strike: float) -> float:
    return max(strike - spot, 0.0)

def payoff_straddle(spot: float, strike: float) -> float:
    return payoff_call(spot, strike) + payoff_put(spot, strike)

    data = yf.download("SPY", period=period, interval=interval, progress=False)
    if data.empty or "Close" not in data:
        raise RuntimeError("Impossible de récupérer les prix SPY")
    return data["Close"]

# Prix de clôture SPY (1 an)
strike_ref = spot_ref  # straddle centré au spot


## Évolution du sous-jacent (SPY)
Historique des clôtures sur un an et repère du spot de référence (dernier close).

In [ ]:
fig, ax = plt.subplots(figsize=(10, 4))
close_spy.plot(ax=ax, color="steelblue", label="SPY close")
ax.axhline(spot_ref, color="crimson", linestyle="--", label=f"Dernier close ≈ {spot_ref:.2f}")
ax.set_title("SPY - clôtures sur 1 an")
ax.set_xlabel("Date")
ax.set_ylabel("Prix")
ax.legend()
plt.show()

## Payoff du straddle long

Le payoff d'un straddle long est la somme du payoff d'un call et d'un put au même strike. La courbe est minimale au strike et croît linéairement en ITM de chaque côté.

In [ ]:
S_grid = np.linspace(spot_ref * 0.5, spot_ref * 1.5, 200)
payoffs = [payoff_straddle(S, strike_ref) for S in S_grid]

fig, ax = plt.subplots(figsize=(8, 5))
ax.plot(S_grid, payoffs, color="navy", label="Payoff straddle long")
ax.axvline(strike_ref, color="gray", linestyle="--", label=f"Strike = {strike_ref:.2f}")
ax.set_xlabel("Spot S")
ax.set_ylabel("Payoff à maturité")
ax.set_title("Straddle long sur SPY : payoff vs spot")
ax.legend()
plt.show()

In [ ]:

import ipywidgets as widgets
from IPython.display import display, Markdown

spot_slider = widgets.FloatSlider(value=spot_ref, min=spot_ref*0.5, max=spot_ref*1.5, step=1.0, description='Spot')
strike_slider = widgets.FloatSlider(value=spot_ref, min=spot_ref*0.5, max=spot_ref*1.5, step=1.0, description='Strike')
output = widgets.Output()

def _update_payoff(change=None):
    with output:
        output.clear_output()
        s = spot_slider.value
        k = strike_slider.value
        p = payoff_straddle(s, k)
        display(Markdown(f"**Spot = {s:.2f}**

- Strike = {k:.2f}
- Payoff straddle = {p:.4f}"))

for sl in (spot_slider, strike_slider):
    sl.observe(_update_payoff, names='value')

_update_payoff()
display(widgets.VBox([spot_slider, strike_slider, output]))
